In [ ]:
from pathlib import Path
target_name = '👾👾👾👾'
reference_path = Path(target_name)

In [ ]:
*(reference_path).glob('*'),

In [ ]:
import pandas as pd

meta = pd.read_csv(reference_path / 'metadata.csv') # site_name smiles

In [ ]:
import pyrosetta
import pyrosetta_help as ph
import logging

logger = ph.configure_logger()
logger.handlers[0].setLevel(logging.ERROR)


extra_options = ph.make_option_string(no_optH=False,
                                      ex1=None,
                                      ex2=None,
                                      #mute='all',
                                      ignore_unrecognized_res=True,
                                      load_PDB_components=False,
                                      ignore_waters=False)
pyrosetta.init(extra_options=extra_options)

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from rdkit_to_params import Params
from typing import List, Dict, Union, Optional, Any, Sequence

logger = logging.getLogger()
if logger.handlers:
    logger.handlers[0].setLevel(logging.ERROR)
    
    
import os
import importlib.util
from types import ModuleType

def import_path(module_path, module_name='custom_module') -> ModuleType:
    """
    A more elegant way that the adding to PATH via sys IMO.
    """
    spec = importlib.util.spec_from_file_location(module_name, module_path)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    return mod


from plip.basic import config

config.NOHYDRO = True
# config.NOPDBCANMAP = False # no idea.
# config.KEEPMOD these are mod residues...


module_path = (Path.home() / 'Coding' / 'Fragment-hit-follow-chemistry' / 'followup' / 'plip.py')
SerialPLIPper = import_path(module_path).SerialPLIPper
    
def load_pose(holo_path: Union[str, Path], smiles: str):
    #pose = ph.parameterized_pose_from_file(holo_path, wanted_ligands={'LIG': smiles}, proximityBonding=True)
    params = Params.from_smiles_w_pdbfile(str(holo_path), smiles=smiles, name='LIG', proximityBonding=True)
    pose = pyrosetta.Pose()
    params.add_residuetype(pose, True)
    pyrosetta.pose_from_file(pose, str(holo_path))
    return pose, params

def lig_details(pose: pyrosetta.Pose, resn='LIG'):
    # this is a weird way of doing but 
    # the Selector approach is glitchy nn-std type set names
    for i, lig in enumerate(pose.residues):
        if lig.name3() == resn:
            lig_num = i+1
            break
    else:
        raise ValueError(f'{resn} absent — unrecognised?')
    #assert lig.name3() == 'LIG', 'Not LIG
    atom_names: List[str] = [lig.atom_name(i+1) for i in range(lig.natoms())]
    lig_resi=pose.pdb_info().number(lig_num)
    lig_chain=pose.pdb_info().chain(lig_num)
    return atom_names, lig_resi, lig_chain

In [ ]:
analysis = {}

for i, row in meta.iterrows():
    if row.crystal_name in analysis:
        continue
    # f'{target_name}-{hit_name}'
    folder = reference_path / 'aligned' / row.crystal_name
    holo_path = folder / f'{row.crystal_name}_bound.pdb'
    try:
        pose, params = load_pose(holo_path, row.smiles)
        pdb_block: str = ph.get_pdbstr(pose)
    except Exception as error:
        print(error.__class__.__name__, error, row.crystal_name)
        continue
    # store
    pose.dump_pdb(str(folder / f'{row.crystal_name}_protonated.pdb'))
    params.dump(str(folder / f'{row.crystal_name}.params'))
    # details
    atom_names, lig_resi, lig_chain = lig_details(pose)
    plipper = SerialPLIPper(ph.get_pdbstr(pose),
                        resn='LIG',
                        resi=lig_resi,
                        chain=lig_chain)
    details = plipper.summarize_interactions(atom_names)
    analysis[row.crystal_name] = dict(resn='LIG',
                                        resi=lig_resi,
                                        chain=lig_chain,
                                        interactions=details)

In [ ]:
import json

with open('hit_interactions.json', 'w') as w:
    json.dump(analysis, w)

In [ ]:
pd.DataFrame([{**dict(crystal=k, resn=d['resn'], chain=d['chain'], intxn_N=di),
               **dd} for k, d in analysis.items() for di, dd in enumerate(d['interactions'])])\
  .to_csv('hit_interactions.csv')

In [ ]:
print('\n'.join([l for l in holo_path.read_text().split('\n') if 'LIG' in l or 'CON' in l]))